In [198]:
import numpy as np
import pandas as pd

import lightgbm as lgb
import pickle

from collections import defaultdict

In [88]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [172]:
df_train, df_test, df_subm = utils.load_dataset()
df_all = utils.merge_dataset(df_train, df_test)

In [207]:
df_train

,cfips,county,state,first_day_of_month,microbusiness_density,active,Unemployment_rate_2019,Unemployment_rate_2020,Unemployment_rate_2021,uprate1,uprate2,uprate3
row_id,,,,,,,,,,,,
1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2.9,5.4,2.8,1.862069,0.518519,0.965517
...,...,...,...,...,...,...,...,...,...,...,...,...
56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,3.0,3.8,3.2,1.266667,0.842105,1.066667
56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,3.0,3.8,3.2,1.266667,0.842105,1.066667
56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,3.0,3.8,3.2,1.266667,0.842105,1.066667


In [179]:
df_housing = pd.read_csv('../input/census_region_housing_starts.csv')

In [185]:
dic = dict()
for states in df_housing['states']:
    states = list(states)
    for state in states:
        dic[state] += 1

KeyError: '['

In [206]:
df_housing

,Unnamed: 0,states,1991-01-01,1991-02-01,1991-03-01,1991-04-01,1991-05-01,1991-06-01,1991-07-01,1991-08-01,...,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01
0,East North Central\n(NSA),"['Indiana', 'Illinois', 'Michigan', 'Ohio', 'W...",100.0,100.91,101.30,101.69,102.32,102.77,102.97,103.14,...,294.98,300.73,307.37,314.00,320.27,321.21,320.22,317.04,318.49,315.83
1,East North Central\n(SA),"['Indiana', 'Illinois', 'Michigan', 'Ohio', 'W...",100.0,100.97,100.92,100.99,101.37,101.50,101.87,102.04,...,298.64,302.12,306.32,310.11,314.36,313.44,313.44,312.30,313.45,313.25
2,East South Central\n(NSA),"['Alabama', 'Kentucky', 'Mississippi', 'Tennes...",100.0,101.01,100.87,100.81,101.14,102.01,101.88,101.72,...,346.44,352.41,360.38,364.05,371.43,376.00,377.04,373.33,376.07,375.03
3,East South Central\n(SA),"['Alabama', 'Kentucky', 'Mississippi', 'Tennes...",100.0,100.61,100.45,100.45,100.75,101.47,101.42,100.86,...,346.35,351.82,358.20,361.01,366.80,370.58,369.65,368.46,372.87,371.81
4,Middle Atlantic\n(NSA),"['New Jersey', 'New York', 'Pennsylvania']",100.0,100.14,100.07,99.59,99.72,99.86,100.33,99.81,...,318.77,324.23,330.05,336.35,341.99,345.47,343.58,338.99,339.85,339.77
5,Middle Atlantic\n(SA),"['New Jersey', 'New York', 'Pennsylvania']",100.0,100.34,99.84,99.29,99.26,98.98,99.25,98.82,...,319.33,325.17,329.25,332.91,335.92,340.29,338.06,335.62,336.60,337.06
6,Mountain\n\n(NSA),"['Arizona', 'Colorado', 'Idaho', 'New Mexico',...",100.0,98.40,100.26,100.30,100.69,102.06,101.53,101.78,...,548.89,565.23,582.83,595.90,603.13,597.84,593.02,578.75,574.88,572.14
7,Mountain\n\n(SA),"['Arizona', 'Colorado', 'Idaho', 'New Mexico',...",100.0,98.78,100.69,100.39,100.50,101.56,101.39,101.26,...,553.76,565.26,578.81,587.24,594.82,590.83,584.45,574.64,570.73,569.76
8,New England\n\n(NSA),"['Connecticut', 'Maine', 'Massachusetts', 'New...",100.0,101.70,101.47,100.97,99.70,99.09,99.08,98.29,...,348.41,355.04,366.17,376.63,383.10,380.67,377.83,377.15,371.87,375.24
9,New England\n\n(SA),"['Connecticut', 'Maine', 'Massachusetts', 'New...",100.0,102.19,101.28,100.97,99.74,98.58,98.69,97.72,...,349.64,357.82,364.11,371.82,376.35,373.86,370.36,371.02,367.41,372.52


In [203]:
dic = defaultdict(int)
for state in df_train['state'].unique():
    for r in range(df_housing.shape[0]):
        if state in df_housing.loc[r, 'states']:
            dic[state] += 1

In [204]:
dic

defaultdict(int,
            {'Alabama': 2,
             'Alaska': 2,
             'Arizona': 2,
             'Arkansas': 2,
             'California': 2,
             'Colorado': 2,
             'Connecticut': 2,
             'Delaware': 2,
             'District of Columbia': 2,
             'Florida': 2,
             'Georgia': 2,
             'Hawaii': 2,
             'Idaho': 2,
             'Illinois': 2,
             'Indiana': 2,
             'Iowa': 2,
             'Kansas': 2,
             'Kentucky': 2,
             'Louisiana': 2,
             'Maine': 2,
             'Maryland': 2,
             'Massachusetts': 2,
             'Michigan': 2,
             'Minnesota': 2,
             'Mississippi': 2,
             'Missouri': 2,
             'Montana': 2,
             'Nebraska': 2,
             'Nevada': 2,
             'New Hampshire': 2,
             'New Jersey': 2,
             'New Mexico': 2,
             'New York': 2,
             'North Carolina': 2,
             